In [ ]:
import chess
import pygame
from pygame.locals import *

# Initialize pygame
pygame.init()

# Set up display
WIDTH, HEIGHT = 800, 800
WINDOW = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption('Play Chess Against AI')

# Load images
PIECES = ['wP', 'wN', 'wB', 'wR', 'wQ', 'wK', 'p', 'n', 'b', 'r', 'q', 'k']
IMAGES = {}
for piece in PIECES:
    
    IMAGES[piece[1:] if piece.startswith('w') else piece] = pygame.transform.scale(pygame.image.load(f'images/{piece}.png'), (100, 100))

# Draw the chess board
def draw_board():
    colors = [pygame.Color(235, 235, 208), pygame.Color(119, 148, 85)]
    for r in range(8):
        for c in range(8):
            color = colors[(r+c) % 2]
            pygame.draw.rect(WINDOW, color, pygame.Rect(c*100, r*100, 100, 100))

# Draw pieces on the board
def draw_pieces(board):
    for r in range(8):
        for c in range(8):
            piece = board.piece_at(chess.square(c, 7-r))
            if piece is not None:
   
                symbol = str(piece)
                try:
                    piece_image = IMAGES[symbol]
                    WINDOW.blit(piece_image, pygame.Rect(c*100, r*100, 100, 100))
                except KeyError:
                    print("KeyError: Could not find image for symbol", symbol)


# Get the AI's move
# Get the AI's move
def get_ai_move(board):
    one_hot = board_to_one_hot(board)
    one_hot = np.expand_dims(one_hot, axis=0)
    prediction = model.predict(one_hot)
    predicted_board = prediction[0]
    
    # Find the index of the move in the predicted board state
    move_index = np.argmax(predicted_board)
    
    # Convert the index to a move
    legal_moves = list(board.legal_moves)
    if move_index < len(legal_moves):
        return legal_moves[move_index]
    else:
        # If the predicted move index exceeds the number of legal moves,
        # return the move at index 0 (just to avoid index out of range error)
        return legal_moves[0] if legal_moves else None

# Main game loop
def main():
    board = chess.Board()
    running = True
    clock = pygame.time.Clock()
    selected_square = None
    player_turn = True

    while running:
        draw_board()
        draw_pieces(board)
        pygame.display.flip()

        for event in pygame.event.get():
            if event.type == QUIT:
                running = False
            elif event.type == MOUSEBUTTONDOWN and player_turn:
                x, y = event.pos
                print("x and y pos",x,y)
                row, col = y // 100, x // 100
                print("row and column",row,col)
                square = chess.square(col, 7-row)

                if selected_square is not None:
                    print(selected_square, square)
                    move = chess.Move(selected_square, square)
                    if move in board.legal_moves:
                        print("Selected move:", move)  # Print the selected move
                        board.push(move)
                        player_turn = False
                        selected_square = None
                    else:
                        print("Illegal move")
                        selected_square = square
                else:
                    print("sel:",square)
                    selected_square = square

        if not player_turn:
            ai_move = get_ai_move(board)
            if ai_move:
                print("AI move:", ai_move)  # Print the AI's move
                board.push(ai_move)
            else:
                print("AI cannot make a legal move")
            player_turn = True

        clock.tick(60)

    pygame.quit()

if __name__ == '__main__':
    main()

